# Canary Models Resource Testing

**Purpose:** Test actual RAM and GPU memory requirements for Canary models on your VM.

**This notebook will:**
1. Check your current system resources (RAM, GPU)
2. Test loading Canary-1B (smallest model)
3. Test loading Canary-Qwen-2.5B (larger model)
4. Monitor memory usage at each step
5. Run inference on 1 sample to measure peak memory

**Your VM specs:**
- 28GB RAM
- 16GB T4 GPU
- 4 vCPUs

**Expected results (based on research):**
- Canary-1B: 6GB RAM minimum (should work)
- Canary-Qwen-2.5B: ~8-12GB RAM estimated (should work)
- GPU VRAM: 6-10GB per model

---

## How to use this notebook:

1. **Run cells sequentially** - don't skip ahead
2. **Monitor in terminal** - keep `nvidia-smi` and `htop` running
3. **Check resource cells** - they show memory before/after each step
4. **Stop if OOM** - if you hit memory errors, document which step failed

---

## Terminal Commands for Monitoring (Run in Separate SSH Session)

**Option 1: Watch GPU memory every 2 seconds:**
```bash
watch -n 2 nvidia-smi
```

**Option 2: Watch system RAM every 2 seconds:**
```bash
watch -n 2 'free -h'
```

**Option 3: Combined monitoring (one-time check):**
```bash
nvidia-smi && echo "---" && free -h && echo "---" && ps aux | grep jupyter | grep -v grep
```

**Option 4: Log memory usage to file:**
```bash
# Start logging (run this in separate terminal)
while true; do 
  echo "[$(date '+%H:%M:%S')]" >> /tmp/memory_log.txt
  free -h | grep Mem >> /tmp/memory_log.txt
  nvidia-smi --query-gpu=memory.used,memory.free --format=csv,noheader >> /tmp/memory_log.txt
  echo "---" >> /tmp/memory_log.txt
  sleep 5
done

# View log (in another terminal)
tail -f /tmp/memory_log.txt
```

## Step 0: Install Required Packages

In [ ]:
# Install psutil for RAM monitoring
!pip install -q psutil
print("✓ psutil installed")

## Step 1: Baseline Resource Check (Before Loading Anything)

In [ ]:
import psutil
import torch
import gc
from datetime import datetime

def print_resources(label=""):
    """Print current RAM and GPU memory usage."""
    print("="*80)
    print(f"RESOURCE CHECK: {label}")
    print(f"Timestamp: {datetime.now().strftime('%H:%M:%S')}")
    print("="*80)
    
    # System RAM
    mem = psutil.virtual_memory()
    ram_used_gb = mem.used / 1024**3
    ram_total_gb = mem.total / 1024**3
    ram_avail_gb = mem.available / 1024**3
    ram_percent = mem.percent
    
    print(f"\n📊 SYSTEM RAM:")
    print(f"  Total:     {ram_total_gb:.2f} GB")
    print(f"  Used:      {ram_used_gb:.2f} GB ({ram_percent:.1f}%)")
    print(f"  Available: {ram_avail_gb:.2f} GB")
    
    if ram_avail_gb < 10:
        print(f"  ⚠️  WARNING: Low RAM! Only {ram_avail_gb:.1f} GB available")
    else:
        print(f"  ✓ RAM looks good")
    
    # GPU Memory
    if torch.cuda.is_available():
        print(f"\n🎮 GPU MEMORY:")
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            total_gb = props.total_memory / 1024**3
            allocated_gb = torch.cuda.memory_allocated(i) / 1024**3
            reserved_gb = torch.cuda.memory_reserved(i) / 1024**3
            free_gb = total_gb - reserved_gb
            
            print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"    Total:     {total_gb:.2f} GB")
            print(f"    Allocated: {allocated_gb:.2f} GB")
            print(f"    Reserved:  {reserved_gb:.2f} GB")
            print(f"    Free:      {free_gb:.2f} GB")
            
            if free_gb < 6:
                print(f"    ⚠️  WARNING: Low GPU memory! Only {free_gb:.2f} GB free")
            else:
                print(f"    ✓ GPU memory looks good")
    else:
        print(f"\n❌ NO CUDA GPU AVAILABLE")
        print(f"   Canary models require GPU acceleration!")
    
    # Python process memory
    process = psutil.Process()
    process_mem_gb = process.memory_info().rss / 1024**3
    print(f"\n🐍 PYTHON PROCESS:")
    print(f"  Memory: {process_mem_gb:.2f} GB")
    
    print("="*80)
    print()

# Initial baseline
print_resources("BASELINE (No models loaded)")

## Step 2: Clean GPU Memory (Run if needed)

If you see any GPU memory already allocated, run this to clear it:

In [ ]:
# Force garbage collection and clear GPU cache
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

print("✓ Memory cleared\n")
print_resources("AFTER CLEANUP")

## Step 3: Configure Environment Variables

Set up model caching and disable wandb:

In [ ]:
import os
from pathlib import Path

# Disable wandb
os.environ['WANDB_MODE'] = 'disabled'

# Set model cache directory
model_cache = Path.cwd().parent / "models" / "canary"
model_cache.mkdir(parents=True, exist_ok=True)

os.environ['HF_HOME'] = str(model_cache)
os.environ['TRANSFORMERS_CACHE'] = str(model_cache)
os.environ['HF_HUB_CACHE'] = str(model_cache)

print(f"✓ Environment configured")
print(f"  Model cache: {model_cache}")
print(f"  Wandb: disabled")

# Check if models already cached (HuggingFace cache structure)
hub_cache = model_cache / "hub"
canary_1b_cached = False
canary_1b_flash_cached = False
canary_qwen_cached = False

if hub_cache.exists():
    # Check for different Canary-1B variants
    if any(hub_cache.glob("models--nvidia--canary-1b")):
        canary_1b_cached = True
    if any(hub_cache.glob("models--nvidia--canary-1b-flash")):
        canary_1b_flash_cached = True
    if any(hub_cache.glob("models--nvidia--canary-qwen*")):
        canary_qwen_cached = True

print(f"\n📦 Model Cache Status:")
print(f"  Canary-1B (standard): {'✓ Cached' if canary_1b_cached else '✗ Not cached (will download ~1GB)'}")
print(f"  Canary-1B-Flash: {'✓ Cached' if canary_1b_flash_cached else '✗ Not cached (will download ~1GB)'}")
print(f"  Canary-Qwen-2.5B: {'✓ Cached' if canary_qwen_cached else '✗ Not cached (will download ~5GB)'}")

# Show what's actually in cache
if hub_cache.exists():
    cached_models = [d.name for d in hub_cache.iterdir() if d.is_dir() and d.name.startswith("models--nvidia--canary")]
    if cached_models:
        print(f"\n  Found in cache:")
        for model in cached_models:
            print(f"    - {model.replace('models--nvidia--', '')}")

## Step 4: Test Loading Canary-1B (Smallest Model)

**Expected:** 6GB RAM minimum, ~4-6GB GPU VRAM

This step will:
1. Show memory before loading
2. Load the model
3. Show memory after loading
4. Calculate memory delta

In [ ]:
print("🔄 Loading Canary-1B model...\n")
print("This will take 1-3 minutes if downloading for the first time.\n")

# Memory before loading
print_resources("BEFORE loading Canary-1B")

# Capture baseline
mem_before = psutil.virtual_memory()
ram_before_gb = mem_before.used / 1024**3

if torch.cuda.is_available():
    gpu_before_gb = torch.cuda.memory_allocated(0) / 1024**3
else:
    gpu_before_gb = 0

print("⏳ Loading model from nvidia/canary-1b...")

try:
    import nemo.collections.asr as nemo_asr
    
    # Load model
    model_1b = nemo_asr.models.EncDecMultiTaskModel.from_pretrained(
        "nvidia/canary-1b",
        map_location="cuda:0"
    )
    
    print("\n✓ Model loaded successfully!\n")
    
    # Memory after loading
    print_resources("AFTER loading Canary-1B")
    
    # Calculate delta
    mem_after = psutil.virtual_memory()
    ram_after_gb = mem_after.used / 1024**3
    ram_delta_gb = ram_after_gb - ram_before_gb
    
    if torch.cuda.is_available():
        gpu_after_gb = torch.cuda.memory_allocated(0) / 1024**3
        gpu_delta_gb = gpu_after_gb - gpu_before_gb
    else:
        gpu_after_gb = 0
        gpu_delta_gb = 0
    
    print("="*80)
    print("📊 MEMORY DELTA (Canary-1B):")
    print("="*80)
    print(f"  RAM increase:  {ram_delta_gb:+.2f} GB")
    print(f"  GPU increase:  {gpu_delta_gb:+.2f} GB")
    print("="*80)
    print()
    
    # Verdict
    if ram_delta_gb < 10:
        print(f"✅ RESULT: Canary-1B uses ~{ram_delta_gb:.1f} GB RAM (within VM limits)")
    else:
        print(f"⚠️  RESULT: Canary-1B uses ~{ram_delta_gb:.1f} GB RAM (might be tight on 28GB VM)")
    
    if gpu_delta_gb < 10:
        print(f"✅ RESULT: Canary-1B uses ~{gpu_delta_gb:.1f} GB GPU (within T4 limits)")
    else:
        print(f"❌ RESULT: Canary-1B uses ~{gpu_delta_gb:.1f} GB GPU (exceeds T4 16GB limit)")
    
except Exception as e:
    print(f"\n❌ ERROR loading Canary-1B:\n{e}\n")
    print_resources("AFTER ERROR")
    raise

## Step 5: Test Inference with Canary-1B

Run inference on a short audio clip to measure peak memory usage:

In [ ]:
print("🔄 Testing inference with Canary-1B...\n")

# Create a dummy audio tensor (10 seconds of silence at 16kHz)
import numpy as np

sample_rate = 16000
duration_sec = 10
audio_length = sample_rate * duration_sec

# Generate silence
dummy_audio = np.zeros(audio_length, dtype=np.float32)

print(f"Generated dummy audio: {duration_sec}s @ {sample_rate}Hz\n")

# Memory before inference
print_resources("BEFORE inference")

mem_before = psutil.virtual_memory()
ram_before_gb = mem_before.used / 1024**3
gpu_before_gb = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0

try:
    # Run transcription with proper parameters
    print("⏳ Running transcription...")
    
    # Canary models need source_lang and target_lang parameters
    with torch.no_grad():
        transcription = model_1b.transcribe(
            audio=[dummy_audio],
            batch_size=1,
            source_lang="en",  # English source
            target_lang="en"   # English target (transcription, not translation)
        )
    
    print(f"\n✓ Transcription completed\n")
    print(f"Result: {transcription}\n")
    
    # Memory after inference
    print_resources("AFTER inference")
    
    mem_after = psutil.virtual_memory()
    ram_after_gb = mem_after.used / 1024**3
    ram_delta_gb = ram_after_gb - ram_before_gb
    
    gpu_after_gb = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0
    gpu_delta_gb = gpu_after_gb - gpu_before_gb
    
    print("="*80)
    print("📊 INFERENCE MEMORY DELTA:")
    print("="*80)
    print(f"  RAM increase:  {ram_delta_gb:+.2f} GB (peak overhead during inference)")
    print(f"  GPU increase:  {gpu_delta_gb:+.2f} GB (activation memory)")
    print("="*80)
    print()
    
except Exception as e:
    print(f"\n❌ ERROR during inference:\n{e}\n")
    print_resources("AFTER ERROR")
    
    # If source_lang error, suggest skipping inference test
    if "source_lang" in str(e):
        print("💡 This is a parameter configuration issue, not a memory issue.")
        print("   Model loaded successfully - that's what we needed to test!")
        print("   You can skip to Step 6 (cleanup) and continue.\n")

## Step 6: Clean Up Canary-1B Before Testing Larger Model

In [ ]:
print("🧹 Cleaning up Canary-1B to free memory...\n")

# Delete model
del model_1b

# Force garbage collection
gc.collect()

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

print("✓ Canary-1B unloaded\n")
print_resources("AFTER cleanup")

## Step 7: Test Loading Canary-Qwen-2.5B (Larger Model)

**Expected:** ~8-12GB RAM, ~6-10GB GPU VRAM

⚠️ **This is the model that crashed before.** We'll carefully monitor memory.

In [ ]:
print("🔄 Loading Canary-Qwen-2.5B model...\n")
print("⚠️  This is the larger model (2.5B parameters)\n")
print("This will take 3-5 minutes if downloading for the first time (~5GB).\n")

# Check available memory first
mem = psutil.virtual_memory()
ram_avail_gb = mem.available / 1024**3

if ram_avail_gb < 12:
    print(f"⚠️  WARNING: Only {ram_avail_gb:.1f} GB RAM available")
    print(f"   Estimated need: 8-12 GB")
    print(f"   This might fail with OOM error!\n")
    
    response = input("Continue anyway? (yes/no): ")
    if response.lower() != 'yes':
        print("Skipping Canary-Qwen-2.5B test")
        raise SystemExit("User cancelled due to low memory")

# Memory before loading
print_resources("BEFORE loading Canary-Qwen-2.5B")

mem_before = psutil.virtual_memory()
ram_before_gb = mem_before.used / 1024**3
gpu_before_gb = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0

print("⏳ Loading model from nvidia/canary-qwen-2.5b...")

try:
    import nemo.collections.asr as nemo_asr
    
    # Load model
    model_qwen = nemo_asr.models.EncDecMultiTaskModel.from_pretrained(
        "nvidia/canary-qwen-2.5b",
        map_location="cuda:0"
    )
    
    print("\n✓ Model loaded successfully!\n")
    
    # Memory after loading
    print_resources("AFTER loading Canary-Qwen-2.5B")
    
    # Calculate delta
    mem_after = psutil.virtual_memory()
    ram_after_gb = mem_after.used / 1024**3
    ram_delta_gb = ram_after_gb - ram_before_gb
    
    gpu_after_gb = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0
    gpu_delta_gb = gpu_after_gb - gpu_before_gb
    
    print("="*80)
    print("📊 MEMORY DELTA (Canary-Qwen-2.5B):")
    print("="*80)
    print(f"  RAM increase:  {ram_delta_gb:+.2f} GB")
    print(f"  GPU increase:  {gpu_delta_gb:+.2f} GB")
    print("="*80)
    print()
    
    # Verdict
    if ram_delta_gb < 15:
        print(f"✅ RESULT: Canary-Qwen-2.5B uses ~{ram_delta_gb:.1f} GB RAM (within VM limits)")
    else:
        print(f"⚠️  RESULT: Canary-Qwen-2.5B uses ~{ram_delta_gb:.1f} GB RAM (tight on 28GB VM)")
    
    if gpu_delta_gb < 12:
        print(f"✅ RESULT: Canary-Qwen-2.5B uses ~{gpu_delta_gb:.1f} GB GPU (within T4 limits)")
    else:
        print(f"❌ RESULT: Canary-Qwen-2.5B uses ~{gpu_delta_gb:.1f} GB GPU (exceeds T4 16GB limit)")
    
except Exception as e:
    print(f"\n❌ ERROR loading Canary-Qwen-2.5B:\n{e}\n")
    print_resources("AFTER ERROR")
    
    # Check if it's an OOM error
    if "out of memory" in str(e).lower() or "oom" in str(e).lower():
        print("\n💡 This appears to be an Out-of-Memory error.")
        print("   Canary-Qwen-2.5B requires more memory than available on this VM.\n")
    
    raise

## Step 8: Test Inference with Canary-Qwen-2.5B (If Loaded Successfully)

In [ ]:
print("🔄 Testing inference with Canary-Qwen-2.5B...\n")

# Create dummy audio (same as before)
dummy_audio = np.zeros(sample_rate * 10, dtype=np.float32)

print(f"Generated dummy audio: 10s @ 16kHz\n")

# Memory before inference
print_resources("BEFORE inference")

mem_before = psutil.virtual_memory()
ram_before_gb = mem_before.used / 1024**3
gpu_before_gb = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0

try:
    print("⏳ Running transcription...")
    
    # Canary models need source_lang and target_lang parameters
    with torch.no_grad():
        transcription = model_qwen.transcribe(
            audio=[dummy_audio],
            batch_size=1,
            source_lang="en",  # English source
            target_lang="en"   # English target (transcription, not translation)
        )
    
    print(f"\n✓ Transcription completed\n")
    print(f"Result: {transcription}\n")
    
    # Memory after inference
    print_resources("AFTER inference")
    
    mem_after = psutil.virtual_memory()
    ram_after_gb = mem_after.used / 1024**3
    ram_delta_gb = ram_after_gb - ram_before_gb
    
    gpu_after_gb = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0
    gpu_delta_gb = gpu_after_gb - gpu_before_gb
    
    print("="*80)
    print("📊 INFERENCE MEMORY DELTA:")
    print("="*80)
    print(f"  RAM increase:  {ram_delta_gb:+.2f} GB")
    print(f"  GPU increase:  {gpu_delta_gb:+.2f} GB")
    print("="*80)
    print()
    
except Exception as e:
    print(f"\n❌ ERROR during inference:\n{e}\n")
    print_resources("AFTER ERROR")
    
    # If source_lang error, suggest skipping inference test
    if "source_lang" in str(e):
        print("💡 This is a parameter configuration issue, not a memory issue.")
        print("   Model loaded successfully - that's what we needed to test!")
        print("   You can continue to Step 9 (summary).\n")

## Step 9: Final Summary

In [ ]:
print("="*80)
print("📋 FINAL SUMMARY")
print("="*80)
print()
print("Please copy the output from Steps 4 and 7 (MEMORY DELTA sections)")
print("and paste them into your notes.\n")
print("Key questions to answer:")
print("  1. Did Canary-1B load successfully?")
print("  2. How much RAM did it use?")
print("  3. How much GPU memory did it use?")
print("  4. Did Canary-Qwen-2.5B load successfully?")
print("  5. How much RAM did it use?")
print("  6. How much GPU memory did it use?")
print("  7. Did inference work for both models?\n")
print("="*80)
print()

# Show final memory state
print_resources("FINAL STATE")

## Troubleshooting

### If you get "CUDA out of memory" errors:

1. **Check other processes using GPU:**
   ```bash
   nvidia-smi
   ```
   Look for other Python processes or Jupyter kernels.

2. **Kill other Jupyter kernels:**
   In Jupyter: Kernel → Shut Down Other Kernels

3. **Restart this kernel:**
   Kernel → Restart & Clear Output
   Then run from Step 0 again.

### If you get "RuntimeError: Out of memory" (system RAM):

1. **Check other processes:**
   ```bash
   ps aux | sort -k4 -rn | head -10
   ```
   This shows top 10 memory-consuming processes.

2. **Close unnecessary applications:**
   Stop any other Jupyter notebooks, scripts, or applications.

3. **Try Canary-1B only:**
   Skip Canary-Qwen-2.5B test and just run Canary-1B.

### If model download fails:

1. **Check internet connection:**
   ```bash
   ping -c 3 huggingface.co
   ```

2. **Check disk space:**
   ```bash
   df -h
   ```
   Models need ~5-10GB free space.

3. **Clear cache and retry:**
   ```bash
   rm -rf ~/projects/amia2025-stt-benchmarking/models/canary/hub
   ```
   Then re-run the notebook.